In [1]:
%matplotlib inline
import sys, os
sys.path.append('../../')
import h5py
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from decimal import Decimal
#from tools import OHLC, Density
#from tools import open_file, products
from tools.instruments import instruments
from tools.display import highchart
from tools.datatools import norm
from IPython.display import clear_output
import time

In [2]:
from sktime.base import load
from sktime.classification.deep_learning.cnn import CNNClassifier

* 모델 로드

In [3]:
period  = 50
modelfile = os.path.join('..','..','data','trend index','models',f'trend_{period}day')
model = load(modelfile)

type: The given save location: ..\..\data\trend index\models\trend_50day
was not found

In [59]:
savefilepath = os.path.join('..','..','data','trend index','trend_index.hdf')
#savefile = h5py.File(savefilepath, 'a')

In [72]:
quotes = instruments.quotes(fields='ohlc')

In [74]:
symbols = list(quotes.columns.levels[0])
for symbol in symbols:
    savefile = h5py.File(savefilepath, 'a')
    print(symbol)
    quote = quotes[symbol].dropna()
    data=[]
    dates = []
    for group in quote.rolling(period):
        if len(group) != period:
            continue
        
        dates.append(group.index.values.max())
        data.append(group.values)
        
    dates = np.array(dates).astype('M8[s]').astype('int64').reshape((len(dates), 1))
    data = np.array(list(map(norm, data)))
    data = np.moveaxis(data, 1, 2)
    
    print("확률 생성중")
    trend = model.predict_proba(data)
    arr = np.concatenate((dates,trend), axis=1)
    
    if symbol not in savefile.keys():
        savefile.create_group(symbol)
    
    savefile[symbol].create_dataset(f"trend{period}", arr.shape, data=arr)
    savefile.close()
    

AD
확률 생성중
2956/2956 [==============================] - 2s 618us/step
BO
확률 생성중
5148/5148 [==============================] - 3s 659us/step
BP
확률 생성중
3927/3927 [==============================] - 3s 656us/step
C
확률 생성중
5220/5220 [==============================] - 3s 617us/step
CC
확률 생성중
4359/4359 [==============================] - 3s 638us/step
CD
확률 생성중
3779/3779 [==============================] - 2s 610us/step
CL
확률 생성중
3260/3260 [==============================] - 2s 620us/step
CT
확률 생성중
4201/4201 [==============================] - 3s 646us/step
DX
확률 생성중
1990/1990 [==============================] - 1s 656us/step
EC
확률 생성중
1948/1948 [==============================] - 1s 671us/step
ES
확률 생성중
2082/2082 [==============================] - 1s 706us/step
FV
확률 생성중
2832/2832 [==============================] - 2s 691us/step
GC
확률 생성중
3952/3952 [==============================] - 3s 650us/step
HG
확률 생성중
5229/5229 [==============================] - 3s 635us/step
HO
확률 생성중
3544/3544 [==============

### Verification

In [77]:
quote = instruments['GC'].quotes(fields='ohlc')
u = quote.TREND(direction='up', period=50)
s = quote.TREND(direction='side', period=50)
d = quote.TREND(direction='down', period=50)

In [78]:
highchart([
    {'data':quote, 'type':'candlestick'},
    {'data':u, 'type':'line','yaxis':'22'},
    {'data':s, 'type':'line','yaxis':'22'},
    {'data':d, 'type':'line','yaxis':'22'}
])